In [1]:
# Imports
import numpy as np #Mathematical tools
import matplotlib.pyplot as plt #Plots charts
import pandas as pd #Import and manage data sets
from sklearn.neural_network import MLPClassifier 
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
# Preprocessing library
from sklearn.preprocessing import Imputer
# Categorical data encoder library
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Splitting the data set into training and testing sets library
from sklearn.model_selection import train_test_split
#Feature scaling library
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [2]:
# Imports part 2

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
# Functions part 1

def best_output(row):
    # se o downloadTime é diferente de 100 para T2 e T3, ambos os targets completam o download
    if ((row.downloadTimeT2!=100)&(row.downloadTimeT3!=100)):
        # o melhor output será aquele para o qual o downloadTime é menor
        if (row.downloadTimeT2<=row.downloadTimeT3):
            return 0
        else:
            return 1
    
    # se o downloadTime é diferente de 100 apenas para um dos targets, só um deles completa o download
    elif ((row.downloadTimeT2!=100)|(row.downloadTimeT3!=100)):
        # o melhor output será aquele para o qual o downloadTime é diferente de 100
        if (row.downloadTimeT2!=100):
            return 0
        else:
            return 1
        
    # se o downloadTime é igual a 100 para T2 e T3, ambos os targets não completam o download
    elif ((row.downloadTimeT2==100)&(row.downloadTimeT3==100)):
        # o melhor output será aquele para o qual o rxBytes é maior
        if (row.rxBytesT2>=row.rxBytesT3):
            return 0
        else:
            return 1
        
def download_complete(row, column):
    if (row[column]==0):
        if(row.downloadTimeT2!=100.0):
            return 1
        else:
            return 0
    else:
        if(row.downloadTimeT3!=100.0):
            return 1
        else: 
            return 0

        
def download_time(row, column):
    if ((row[column]==0)&(row.downloadTimeT2<100)):
        return row.downloadTimeT2
    elif ((row[column]==1)&(row.downloadTimeT3<100)):
        return row.downloadTimeT3

    
def throughput(row, column):
    if (row[column]==0):
        tp = (row.rxBytesT2/row.downloadTimeT2)*8/1e6
        return tp
    else:
        tp = (row.rxBytesT3/row.downloadTimeT3)*8/1e6
        return tp
    
def optimum_choice(row, column):
    if (row[column]==row.best_output):
        return 1
    else:
        return 0
    
def a2a4rsrp(row):
    if (row.rsrp2>=row.rsrp3):
        return 0
    else:
        return 1

In [12]:
# Functions part 2
def percentageOfErrors(y_real, y_predicted):
    p= ((y_predicted != y_real).sum()/ y_real.size)
    return p 

def applyingClassifier (classifier,x_test, x_train, y_test, y_train):
    classifier.fit(x_train, y_train)
    # Predicting the test set results
    y_pred = classifier.predict(x_test)

    # Evaluation of the model
    # Making the confusion matrix
    #cm = confusion_matrix(y_test, y_pred)
    p=percentageOfErrors(y_real=y_test, y_predicted=y_pred)
    print(p)
    return p

def gettingClassifierConfiguration(classifier, param_test,x_train, y_train ):
    conf=GridSearchCV(estimator=classifier, param_grid=param_test, scoring='roc_auc', cv=10, n_jobs=-1)
    conf.fit(x_train, y_train)
    print(conf.best_estimator_)
    return conf.best_estimator_

def gettingAllClassifiersConfigurations(x_train, y_train, randomState):
    # All test parameters
    param_test_knn = [{'n_neighbors':[(i) for i in range(5,31)], 'p':[1,2],'weights':['uniform', 'distance'],
                  'algorithm':['auto', 'brute']},{'n_neighbors':[(i) for i in range(1,21)], 'p':[1,2],
                 'weights':['uniform', 'distance'],
                  'algorithm':['ball_tree', 'kd_tree'], 'leaf_size':[10,20,30,40,50,60,70, 80,90,100]} ]
    param_test_svm =[{'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'degree':[(i) for i in range(1,21)],'kernel':['poly'],'decision_function_shape':['ovo','ovr'],
                  'shrinking':[True, False]},
                 {'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'kernel':['linear'],'decision_function_shape':['ovo','ovr'],
                  'shrinking':[True, False]},
              {'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'kernel':['rbf', 'sigmoid'],'gamma':[1,0.5,0.1, 0.01,0.001, 0.0001, 0.0000001],
                 'decision_function_shape':['ovo','ovr'], 'shrinking':[True, False]}] 
    # Creating the DT classifier parameters
    param_test_dt=[{'criterion': ['gini', 'entropy'], 'splitter':['best', 'random'], 
                    'max_features':[None,'auto', 'sqrt', 'log2'], 'class_weight':['balanced', None]}]    
   
    # Creating the RF classifier parameters
    param_test_rf = {'n_estimators':[(i) for i in range(5,31)], 'criterion': ['gini', 'entropy'], 
                     'max_features':[None,'auto', 'sqrt', 'log2'], 'class_weight':['balanced', None], 
                     'verbose':[0], 'n_jobs':[-1], 'bootstrap':[True, False]}
    # All classifiers
    classifier_knn=KNeighborsClassifier(metric = 'minkowski')
    classifier_svm=SVC(random_state=randomState)
    classifier_dt=DecisionTreeClassifier(random_state=randomState)
    classifier_rf=RandomForestClassifier(random_state=randomState)
    
    #Finding the best configurations
    print("Best parameters for KNN Classifier")
    conf_knn=gettingClassifierConfiguration(classifier=classifier_knn, param_test=param_test_knn, 
                                            x_train=x_train, y_train=y_train)
    
    print("Best parameters for SVM Classifier")
    conf_svm=gettingClassifierConfiguration(classifier=classifier_svm, param_test=param_test_svm,
                                            x_train=x_train, y_train=y_train)
    print("Best parameters for DT Classifier")
    conf_dt=gettingClassifierConfiguration(classifier=classifier_dt, param_test=param_test_dt,
                                           x_train=x_train, y_train=y_train)
    print("Best parameters for RF Classifier")
    conf_rf=gettingClassifierConfiguration(classifier=classifier_rf, param_test=param_test_rf,
                                           x_train=x_train, y_train=y_train)
    

    
    conf=[conf_knn,conf_svm, conf_dt, conf_rf]
    return conf
    
    
    
def applyingAllClassifiers (classifiers,numberOfRandomStates,x, y):   
    # Initializing the variables
    p_svm=0
    p_dt=0
    p_rf=0
    p_knn=0
    for (i) in range(0,numberOfRandomStates):
        r1=np.random.randint(1,10001)
        r2=np.random.randint(1,10001)
        # Splitting test data and train data
        x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = r1)
        # Feature scaling (normalizing or stadardization of the scales)
        # Helps the conversion of the algorithm
        sc_X = StandardScaler()
        x_train = sc_X.fit_transform(x_train)
        x_test = sc_X.transform(x_test) # There is no need to fit after the training set is fit        
        print("{}: RandomState r1={} and r2={}".format(i, r1,r2))
        # Fitting the KNN to the training set
        p_knn=p_knn+applyingClassifier(classifier=classifiers[0],x_test=x_test, x_train=x_train, y_test=y_test, y_train= y_train)
        # Fitting the SVM to the training set
        classifiers[1].randomState=r2
        p_svm=p_svm+applyingClassifier(classifier=classifiers[1],x_test=x_test,x_train=x_train,y_test=y_test,y_train=y_train)
        # Fitting the DT to the training set
        classifiers[2].randomState=r2
        p_dt=p_dt+applyingClassifier(classifier=classifiers[2],x_test=x_test,x_train=x_train,y_test=y_test,y_train=y_train)
        # Fitting the RF to the training set
        classifiers[3].randomState=r2
        p_rf=p_rf+applyingClassifier(classifier=classifiers[3],x_test=x_test,x_train=x_train,y_test=y_test,y_train=y_train)
    p_knn=p_knn/numberOfRandomStates
    p_svm=p_svm/numberOfRandomStates
    p_dt=p_dt/numberOfRandomStates
    p_rf=p_rf/numberOfRandomStates
    
    p=np.array([p_knn, p_svm, p_dt, p_rf])
    return p   
    

# Sem shadowing

In [5]:
# Importing data

# importando os datasets
t2 = pd.read_csv('resultados/t2_OkumuraHata_Modificado', delimiter='\t')
t3 = pd.read_csv('resultados/t3_OkumuraHata_Modificado', delimiter='\t')

# garantindo que utilizaremos apenas as sementes presentes nos dois datasets
t2 = t2[t2.nRun.isin(t3.nRun)]
t3 = t3[t3.nRun.isin(t2.nRun)]
t2 = t2.reset_index(drop=True)
t3 = t3.reset_index(drop=True)

# combinando os datasets
data = t2
data = data.drop(['targetCellId', 'downloadTime', 'rxBytes'], axis=1)
data['downloadTimeT2'] = t2.downloadTime
data['downloadTimeT3'] = t3.downloadTime
data['rxBytesT2'] = t2.rxBytes
data['rxBytesT3'] = t3.rxBytes

data['best_output'] = data.apply(best_output, axis=1)
data.head()

# Splitting the data between independent variables and dependent variable
y = data['best_output']
x = data[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1','previousrsrq1','previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]

# Splitting test data and train data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
# Feature scaling (normalizing or stadardization of the scales)
# Helps the conversion of the algorithm
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test) # There is no need to fit after the training set is fit

In [13]:
conf=gettingAllClassifiersConfigurations(x_train=x_train, y_train=y_train, randomState=0)

# Setting the classifiers
classifier_knn=conf[0]
classifier_svm=conf[1]
classifier_dt=conf[2]
classifier_rf=conf[3]
#classifier_knn=KNeighborsClassifier(algorithm='auto',n_neighbors=5, p=1, weights='uniform')
#classifier_svm=SVC(C=200, decision_function_shape='ovo', degree=3, kernel='poly', shrinking=True)
#classifier_dt=DecisionTreeClassifier(class_weight= 'balanced', criterion= 'gini', max_features= None, splitter= 'best')
#classifier_rf=RandomForestClassifier(bootstrap= True, class_weight='balanced', criterion= 'gini', 
#                                     max_features= None, n_estimators= 2, n_jobs= -1, verbose= 0)


classifiers=[classifier_knn,classifier_svm,classifier_dt,classifier_rf]

Best parameters for KNN Classifier
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=1,
           weights='uniform')
Best parameters for SVM Classifier
SVC(C=200, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)
Best parameters for DT Classifier
DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')
Best parameters for RF Classifier
RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features

In [16]:
# Applying the classifiers
maxRange=1000
p = applyingAllClassifiers(numberOfRandomStates=maxRange, x=x, y=y, classifiers=classifiers)

0: RandomState r1=7946 and r2=3975
0.005
0.005
0.0
0.0
1: RandomState r1=5101 and r2=8312
0.0
0.0
0.005
0.005
2: RandomState r1=7683 and r2=669
0.005
0.01
0.005
0.005
3: RandomState r1=9539 and r2=777
0.0
0.0
0.0
0.0
4: RandomState r1=6592 and r2=4210
0.005
0.005
0.015
0.015
5: RandomState r1=6828 and r2=5861
0.0
0.0
0.0
0.0
6: RandomState r1=1164 and r2=9421
0.0
0.0
0.0
0.0
7: RandomState r1=581 and r2=1112
0.005
0.005
0.005
0.01
8: RandomState r1=4436 and r2=5352
0.005
0.005
0.005
0.005
9: RandomState r1=2000 and r2=7834
0.0
0.0
0.01
0.01
10: RandomState r1=8948 and r2=8059
0.005
0.01
0.005
0.005
11: RandomState r1=993 and r2=6741
0.0
0.0
0.0
0.0
12: RandomState r1=6534 and r2=7523
0.0
0.0
0.005
0.005
13: RandomState r1=8258 and r2=483
0.0
0.0
0.015
0.015
14: RandomState r1=5268 and r2=5452
0.005
0.005
0.0
0.0
15: RandomState r1=8601 and r2=7956
0.005
0.005
0.005
0.005
16: RandomState r1=9479 and r2=6373
0.005
0.005
0.005
0.0
17: RandomState r1=4797 and r2=9702
0.005
0.005
0.005
0.01

0.01
147: RandomState r1=5134 and r2=9126
0.005
0.005
0.005
0.005
148: RandomState r1=2086 and r2=9000
0.0
0.0
0.005
0.005
149: RandomState r1=9261 and r2=1877
0.005
0.005
0.005
0.005
150: RandomState r1=244 and r2=1537
0.005
0.005
0.0
0.0
151: RandomState r1=1633 and r2=4969
0.005
0.005
0.01
0.005
152: RandomState r1=8055 and r2=9814
0.0
0.0
0.01
0.0
153: RandomState r1=1604 and r2=3010
0.005
0.005
0.005
0.005
154: RandomState r1=6281 and r2=812
0.0
0.005
0.0
0.0
155: RandomState r1=3094 and r2=6634
0.0
0.0
0.005
0.005
156: RandomState r1=4651 and r2=6993
0.0
0.005
0.0
0.0
157: RandomState r1=1098 and r2=4812
0.0
0.0
0.0
0.0
158: RandomState r1=1549 and r2=3746
0.005
0.01
0.005
0.005
159: RandomState r1=2655 and r2=8700
0.0
0.005
0.005
0.005
160: RandomState r1=6244 and r2=201
0.0
0.0
0.01
0.005
161: RandomState r1=2991 and r2=3623
0.0
0.005
0.0
0.0
162: RandomState r1=6702 and r2=219
0.01
0.01
0.005
0.005
163: RandomState r1=4518 and r2=3176
0.005
0.005
0.005
0.005
164: RandomState r

0.0
290: RandomState r1=5346 and r2=3042
0.005
0.005
0.005
0.005
291: RandomState r1=998 and r2=5954
0.0
0.005
0.0
0.0
292: RandomState r1=8446 and r2=9528
0.005
0.01
0.005
0.005
293: RandomState r1=7595 and r2=1960
0.005
0.005
0.0
0.0
294: RandomState r1=7279 and r2=4025
0.005
0.005
0.005
0.005
295: RandomState r1=5202 and r2=5613
0.0
0.0
0.0
0.0
296: RandomState r1=852 and r2=109
0.01
0.01
0.0
0.0
297: RandomState r1=8584 and r2=8303
0.0
0.0
0.0
0.0
298: RandomState r1=6343 and r2=2735
0.0
0.0
0.0
0.0
299: RandomState r1=5083 and r2=129
0.01
0.01
0.005
0.005
300: RandomState r1=9890 and r2=8666
0.0
0.0
0.0
0.0
301: RandomState r1=6970 and r2=6052
0.005
0.005
0.005
0.005
302: RandomState r1=7438 and r2=565
0.005
0.005
0.005
0.01
303: RandomState r1=7632 and r2=7359
0.005
0.005
0.005
0.005
304: RandomState r1=7880 and r2=8447
0.01
0.01
0.01
0.005
305: RandomState r1=7727 and r2=9732
0.005
0.005
0.005
0.01
306: RandomState r1=3367 and r2=7225
0.01
0.01
0.005
0.015
307: RandomState r1=68

0.005
436: RandomState r1=8041 and r2=8873
0.0
0.0
0.005
0.005
437: RandomState r1=1868 and r2=1694
0.0
0.0
0.005
0.005
438: RandomState r1=6627 and r2=7114
0.0
0.0
0.0
0.0
439: RandomState r1=1877 and r2=1715
0.0
0.0
0.0
0.0
440: RandomState r1=3429 and r2=6488
0.005
0.01
0.005
0.005
441: RandomState r1=4792 and r2=1971
0.0
0.0
0.0
0.0
442: RandomState r1=7550 and r2=8502
0.0
0.0
0.0
0.0
443: RandomState r1=7094 and r2=9088
0.0
0.005
0.0
0.005
444: RandomState r1=7998 and r2=4005
0.0
0.005
0.005
0.0
445: RandomState r1=7 and r2=2061
0.005
0.005
0.005
0.005
446: RandomState r1=9457 and r2=5291
0.0
0.0
0.0
0.0
447: RandomState r1=5464 and r2=7234
0.005
0.005
0.005
0.005
448: RandomState r1=3507 and r2=6483
0.005
0.005
0.0
0.005
449: RandomState r1=2863 and r2=5453
0.005
0.005
0.005
0.005
450: RandomState r1=3284 and r2=632
0.0
0.005
0.01
0.01
451: RandomState r1=9776 and r2=1362
0.005
0.01
0.005
0.005
452: RandomState r1=5808 and r2=5958
0.005
0.005
0.005
0.005
453: RandomState r1=2656 

0.0
582: RandomState r1=5850 and r2=5261
0.015
0.015
0.01
0.015
583: RandomState r1=4535 and r2=5554
0.0
0.0
0.0
0.0
584: RandomState r1=2223 and r2=278
0.005
0.005
0.005
0.005
585: RandomState r1=4627 and r2=2324
0.0
0.0
0.005
0.005
586: RandomState r1=9677 and r2=1158
0.005
0.005
0.005
0.0
587: RandomState r1=3254 and r2=7848
0.0
0.0
0.0
0.0
588: RandomState r1=3153 and r2=4780
0.005
0.005
0.005
0.005
589: RandomState r1=3631 and r2=3288
0.005
0.005
0.0
0.0
590: RandomState r1=7468 and r2=5837
0.0
0.0
0.0
0.0
591: RandomState r1=1774 and r2=322
0.005
0.005
0.0
0.0
592: RandomState r1=6041 and r2=9986
0.0
0.005
0.005
0.005
593: RandomState r1=9160 and r2=9870
0.0
0.005
0.01
0.01
594: RandomState r1=6497 and r2=4899
0.0
0.0
0.005
0.005
595: RandomState r1=8654 and r2=8071
0.01
0.01
0.005
0.005
596: RandomState r1=9982 and r2=9618
0.0
0.0
0.0
0.0
597: RandomState r1=7687 and r2=3601
0.0
0.0
0.0
0.0
598: RandomState r1=280 and r2=7021
0.005
0.005
0.0
0.0
599: RandomState r1=8561 and r2=1

0.005
727: RandomState r1=5247 and r2=476
0.005
0.005
0.0
0.0
728: RandomState r1=7363 and r2=3174
0.0
0.0
0.005
0.005
729: RandomState r1=7991 and r2=9541
0.005
0.005
0.0
0.0
730: RandomState r1=8267 and r2=5989
0.005
0.005
0.005
0.005
731: RandomState r1=1318 and r2=907
0.005
0.005
0.005
0.005
732: RandomState r1=1638 and r2=3001
0.005
0.005
0.0
0.0
733: RandomState r1=5638 and r2=3294
0.0
0.0
0.005
0.005
734: RandomState r1=6672 and r2=7084
0.0
0.005
0.0
0.0
735: RandomState r1=7405 and r2=9370
0.005
0.005
0.005
0.005
736: RandomState r1=6391 and r2=5064
0.0
0.0
0.0
0.0
737: RandomState r1=4618 and r2=740
0.01
0.015
0.015
0.01
738: RandomState r1=881 and r2=7779
0.005
0.005
0.0
0.0
739: RandomState r1=7117 and r2=8644
0.005
0.01
0.005
0.01
740: RandomState r1=819 and r2=3110
0.005
0.005
0.005
0.005
741: RandomState r1=8075 and r2=6935
0.0
0.0
0.0
0.0
742: RandomState r1=2298 and r2=20
0.005
0.005
0.0
0.0
743: RandomState r1=6427 and r2=1025
0.0
0.0
0.0
0.0
744: RandomState r1=4682 a

0.0
872: RandomState r1=3778 and r2=4362
0.005
0.005
0.0
0.0
873: RandomState r1=1317 and r2=4928
0.005
0.005
0.005
0.005
874: RandomState r1=8517 and r2=4622
0.0
0.0
0.005
0.005
875: RandomState r1=3065 and r2=70
0.0
0.0
0.005
0.005
876: RandomState r1=7303 and r2=5208
0.0
0.0
0.0
0.0
877: RandomState r1=5566 and r2=854
0.015
0.015
0.005
0.005
878: RandomState r1=7142 and r2=5940
0.01
0.01
0.005
0.005
879: RandomState r1=5627 and r2=794
0.0
0.0
0.0
0.0
880: RandomState r1=4537 and r2=8824
0.0
0.0
0.0
0.0
881: RandomState r1=1588 and r2=9733
0.0
0.0
0.0
0.0
882: RandomState r1=7412 and r2=7333
0.0
0.0
0.005
0.005
883: RandomState r1=1621 and r2=4017
0.0
0.005
0.005
0.005
884: RandomState r1=2226 and r2=1871
0.005
0.005
0.005
0.005
885: RandomState r1=4811 and r2=7619
0.0
0.0
0.0
0.0
886: RandomState r1=5910 and r2=8848
0.0
0.0
0.0
0.0
887: RandomState r1=6170 and r2=9956
0.005
0.005
0.005
0.005
888: RandomState r1=8317 and r2=1470
0.005
0.005
0.0
0.0
889: RandomState r1=8787 and r2=185

In [17]:
print('Results without shadowing:')
1 - p

Results without shadowing:


array([ 0.996835,  0.99601 ,  0.99669 ,  0.996755])

# Com shadowing

In [18]:
# Importing data

# importando os datasets
t2 = pd.read_csv('resultados/t2_OhBuildings_ComShadowing_Modificado', delimiter='\t')
t3 = pd.read_csv('resultados/t3_OhBuildings_ComShadowing_Modificado', delimiter='\t')

# garantindo que utilizaremos apenas as sementes presentes nos dois datasets
t2 = t2[t2.nRun.isin(t3.nRun)]
t3 = t3[t3.nRun.isin(t2.nRun)]
t2 = t2.reset_index(drop=True)
t3 = t3.reset_index(drop=True)

# combinando os datasets
data = t2
data = data.drop(['targetCellId', 'downloadTime', 'rxBytes'], axis=1)
data['downloadTimeT2'] = t2.downloadTime
data['downloadTimeT3'] = t3.downloadTime
data['rxBytesT2'] = t2.rxBytes
data['rxBytesT3'] = t3.rxBytes

data['best_output'] = data.apply(best_output, axis=1)
data.head()

# Splitting the data between independent variables and dependent variable
y = data['best_output']
x = data[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1','previousrsrq1','previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]

# Splitting test data and train data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
# Feature scaling (normalizing or stadardization of the scales)
# Helps the conversion of the algorithm
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test) # There is no need to fit after the training set is fit

In [19]:
conf=gettingAllClassifiersConfigurations(x_train=x_train, y_train=y_train, randomState=0)

# Setting the classifiers
classifier_knn=conf[0]
classifier_svm=conf[1]
classifier_dt=conf[2]
classifier_rf=conf[3]
#classifier_knn=KNeighborsClassifier(algorithm='auto',n_neighbors=5, p=1, weights='uniform')
#classifier_svm=SVC(C=200, decision_function_shape='ovo', degree=3, kernel='poly', shrinking=True)
#classifier_dt=DecisionTreeClassifier(class_weight= 'balanced', criterion= 'gini', max_features= None, splitter= 'best')
#classifier_rf=RandomForestClassifier(bootstrap= True, class_weight='balanced', criterion= 'gini', 
#                                     max_features= None, n_estimators= 2, n_jobs= -1, verbose= 0)


classifiers=[classifier_knn,classifier_svm,classifier_dt,classifier_rf]

Best parameters for KNN Classifier
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=22, p=2,
           weights='uniform')
Best parameters for SVM Classifier
SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)
Best parameters for DT Classifier
DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')
Best parameters for RF Classifier
RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features

In [20]:
# Applying the classifiers
maxRange=1000
p = applyingAllClassifiers(numberOfRandomStates=maxRange, x=x, y=y, classifiers=classifiers)

0: RandomState r1=4923 and r2=9422
0.345205479452
0.180821917808
0.386301369863
0.367123287671
1: RandomState r1=2953 and r2=7094
0.364383561644
0.216438356164
0.397260273973
0.356164383562
2: RandomState r1=4777 and r2=9373
0.41095890411
0.169863013699
0.375342465753
0.386301369863
3: RandomState r1=4788 and r2=7436
0.350684931507
0.216438356164
0.347945205479
0.372602739726
4: RandomState r1=4965 and r2=6691
0.361643835616
0.2
0.375342465753
0.378082191781
5: RandomState r1=3007 and r2=629
0.347945205479
0.208219178082
0.356164383562
0.364383561644
6: RandomState r1=2529 and r2=6096
0.342465753425
0.232876712329
0.32602739726
0.328767123288
7: RandomState r1=7170 and r2=805
0.331506849315
0.180821917808
0.38904109589
0.328767123288
8: RandomState r1=9899 and r2=3677
0.328767123288
0.197260273973
0.397260273973
0.342465753425
9: RandomState r1=7905 and r2=2106
0.421917808219
0.18904109589
0.375342465753
0.402739726027
10: RandomState r1=8442 and r2=2387
0.353424657534
0.191780821918
0

0.347945205479
87: RandomState r1=5294 and r2=5110
0.33698630137
0.167123287671
0.391780821918
0.347945205479
88: RandomState r1=2662 and r2=6135
0.358904109589
0.153424657534
0.361643835616
0.353424657534
89: RandomState r1=5265 and r2=1666
0.334246575342
0.180821917808
0.41095890411
0.364383561644
90: RandomState r1=5851 and r2=7852
0.361643835616
0.18904109589
0.383561643836
0.358904109589
91: RandomState r1=3726 and r2=7605
0.339726027397
0.191780821918
0.350684931507
0.353424657534
92: RandomState r1=8394 and r2=6455
0.386301369863
0.186301369863
0.372602739726
0.323287671233
93: RandomState r1=3973 and r2=8970
0.323287671233
0.216438356164
0.369863013699
0.347945205479
94: RandomState r1=6710 and r2=9944
0.375342465753
0.180821917808
0.350684931507
0.342465753425
95: RandomState r1=9217 and r2=9631
0.375342465753
0.202739726027
0.358904109589
0.386301369863
96: RandomState r1=1129 and r2=9144
0.331506849315
0.202739726027
0.378082191781
0.380821917808
97: RandomState r1=5052 and 

0.197260273973
0.416438356164
0.334246575342
173: RandomState r1=334 and r2=7288
0.353424657534
0.235616438356
0.342465753425
0.350684931507
174: RandomState r1=1979 and r2=9578
0.402739726027
0.164383561644
0.38904109589
0.342465753425
175: RandomState r1=5798 and r2=296
0.361643835616
0.224657534247
0.361643835616
0.358904109589
176: RandomState r1=7277 and r2=1832
0.295890410959
0.164383561644
0.391780821918
0.304109589041
177: RandomState r1=6135 and r2=4774
0.353424657534
0.205479452055
0.394520547945
0.364383561644
178: RandomState r1=2045 and r2=8687
0.380821917808
0.197260273973
0.378082191781
0.356164383562
179: RandomState r1=6474 and r2=5319
0.33698630137
0.175342465753
0.295890410959
0.323287671233
180: RandomState r1=6452 and r2=3656
0.345205479452
0.18904109589
0.331506849315
0.32602739726
181: RandomState r1=6202 and r2=9291
0.347945205479
0.208219178082
0.378082191781
0.367123287671
182: RandomState r1=5295 and r2=6862
0.342465753425
0.172602739726
0.33698630137
0.34246

0.175342465753
0.345205479452
0.361643835616
259: RandomState r1=7350 and r2=9398
0.342465753425
0.216438356164
0.386301369863
0.356164383562
260: RandomState r1=8166 and r2=6828
0.320547945205
0.205479452055
0.38904109589
0.334246575342
261: RandomState r1=8693 and r2=4580
0.315068493151
0.224657534247
0.38904109589
0.315068493151
262: RandomState r1=6648 and r2=4202
0.358904109589
0.208219178082
0.380821917808
0.358904109589
263: RandomState r1=8966 and r2=16
0.33698630137
0.178082191781
0.369863013699
0.372602739726
264: RandomState r1=1984 and r2=8709
0.350684931507
0.186301369863
0.375342465753
0.345205479452
265: RandomState r1=3040 and r2=5214
0.353424657534
0.2
0.378082191781
0.386301369863
266: RandomState r1=9098 and r2=406
0.331506849315
0.186301369863
0.378082191781
0.32602739726
267: RandomState r1=9635 and r2=5472
0.375342465753
0.186301369863
0.402739726027
0.353424657534
268: RandomState r1=866 and r2=5364
0.33698630137
0.153424657534
0.290410958904
0.290410958904
269: 

0.194520547945
0.353424657534
0.353424657534
345: RandomState r1=5970 and r2=6600
0.378082191781
0.186301369863
0.347945205479
0.347945205479
346: RandomState r1=6039 and r2=9182
0.353424657534
0.180821917808
0.421917808219
0.356164383562
347: RandomState r1=7753 and r2=2330
0.342465753425
0.180821917808
0.356164383562
0.345205479452
348: RandomState r1=4562 and r2=1062
0.323287671233
0.180821917808
0.435616438356
0.342465753425
349: RandomState r1=7412 and r2=713
0.372602739726
0.202739726027
0.347945205479
0.353424657534
350: RandomState r1=9924 and r2=5594
0.33698630137
0.180821917808
0.350684931507
0.320547945205
351: RandomState r1=7364 and r2=9750
0.358904109589
0.180821917808
0.361643835616
0.361643835616
352: RandomState r1=5920 and r2=1455
0.364383561644
0.194520547945
0.4
0.350684931507
353: RandomState r1=8821 and r2=4701
0.345205479452
0.186301369863
0.369863013699
0.331506849315
354: RandomState r1=3410 and r2=3236
0.391780821918
0.164383561644
0.402739726027
0.34520547945

...

0.180821917808
0.358904109589
0.331506849315
604: RandomState r1=4326 and r2=6782
0.364383561644
0.197260273973
0.367123287671
0.383561643836
605: RandomState r1=4464 and r2=1508
0.328767123288
0.18904109589
0.328767123288
0.32602739726
606: RandomState r1=2250 and r2=385
0.328767123288
0.169863013699
0.353424657534
0.342465753425
607: RandomState r1=4779 and r2=2545
0.32602739726
0.21095890411
0.397260273973
0.364383561644
608: RandomState r1=966 and r2=3142
0.317808219178
0.194520547945
0.391780821918
0.323287671233
609: RandomState r1=8267 and r2=735
0.380821917808
0.2
0.4
0.369863013699
610: RandomState r1=786 and r2=3906
0.361643835616
0.197260273973
0.345205479452
0.345205479452
611: RandomState r1=9052 and r2=7641
0.380821917808
0.219178082192
0.372602739726
0.358904109589
612: RandomState r1=6107 and r2=433
0.33698630137
0.164383561644
0.367123287671
0.304109589041
613: RandomState r1=6845 and r2=6400
0.358904109589
0.221917808219
0.369863013699
0.383561643836
614: RandomState 

0.216438356164
0.4
0.383561643836
690: RandomState r1=3874 and r2=8922
0.317808219178
0.186301369863
0.402739726027
0.345205479452
691: RandomState r1=1269 and r2=5602
0.367123287671
0.227397260274
0.413698630137
0.350684931507
692: RandomState r1=5017 and r2=9961
0.331506849315
0.2
0.386301369863
0.334246575342
693: RandomState r1=6874 and r2=5524
0.38904109589
0.219178082192
0.397260273973
0.380821917808
694: RandomState r1=8295 and r2=383
0.334246575342
0.150684931507
0.367123287671
0.339726027397
695: RandomState r1=2822 and r2=7899
0.38904109589
0.208219178082
0.41095890411
0.38904109589
696: RandomState r1=634 and r2=632
0.375342465753
0.208219178082
0.394520547945
0.33698630137
697: RandomState r1=1299 and r2=3919
0.372602739726
0.2
0.383561643836
0.356164383562
698: RandomState r1=9321 and r2=2008
0.380821917808
0.164383561644
0.375342465753
0.356164383562
699: RandomState r1=4457 and r2=795
0.32602739726
0.183561643836
0.358904109589
0.331506849315
700: RandomState r1=9903 and

0.369863013699
776: RandomState r1=1991 and r2=7292
0.320547945205
0.175342465753
0.312328767123
0.32602739726
777: RandomState r1=5471 and r2=5265
0.320547945205
0.18904109589
0.345205479452
0.309589041096
778: RandomState r1=3860 and r2=5671
0.353424657534
0.153424657534
0.383561643836
0.353424657534
779: RandomState r1=8505 and r2=8513
0.383561643836
0.153424657534
0.397260273973
0.372602739726
780: RandomState r1=4070 and r2=9555
0.380821917808
0.172602739726
0.350684931507
0.372602739726
781: RandomState r1=6549 and r2=36
0.339726027397
0.178082191781
0.391780821918
0.358904109589
782: RandomState r1=4307 and r2=3233
0.361643835616
0.208219178082
0.315068493151
0.33698630137
783: RandomState r1=9778 and r2=4617
0.33698630137
0.208219178082
0.394520547945
0.323287671233
784: RandomState r1=542 and r2=8467
0.33698630137
0.2
0.38904109589
0.309589041096
785: RandomState r1=5498 and r2=7001
0.358904109589
0.194520547945
0.419178082192
0.342465753425
786: RandomState r1=4715 and r2=670

0.334246575342
862: RandomState r1=5147 and r2=6279
0.353424657534
0.172602739726
0.342465753425
0.304109589041
863: RandomState r1=9182 and r2=8660
0.358904109589
0.197260273973
0.372602739726
0.315068493151
864: RandomState r1=2088 and r2=4976
0.350684931507
0.224657534247
0.41095890411
0.364383561644
865: RandomState r1=5315 and r2=3204
0.356164383562
0.194520547945
0.356164383562
0.317808219178
866: RandomState r1=9449 and r2=2771
0.361643835616
0.197260273973
0.339726027397
0.32602739726
867: RandomState r1=6576 and r2=2086
0.361643835616
0.186301369863
0.356164383562
0.369863013699
868: RandomState r1=422 and r2=2845
0.353424657534
0.178082191781
0.369863013699
0.342465753425
869: RandomState r1=5747 and r2=1065
0.380821917808
0.202739726027
0.369863013699
0.361643835616
870: RandomState r1=1872 and r2=4613
0.361643835616
0.169863013699
0.347945205479
0.345205479452
871: RandomState r1=9532 and r2=2872
0.38904109589
0.167123287671
0.386301369863
0.350684931507
872: RandomState r1

0.202739726027
0.347945205479
0.353424657534
948: RandomState r1=3112 and r2=4799
0.369863013699
0.254794520548
0.386301369863
0.408219178082
949: RandomState r1=660 and r2=1243
0.383561643836
0.216438356164
0.361643835616
0.367123287671
950: RandomState r1=1332 and r2=8123
0.320547945205
0.224657534247
0.361643835616
0.361643835616
951: RandomState r1=1778 and r2=1560
0.342465753425
0.147945205479
0.353424657534
0.298630136986
952: RandomState r1=4452 and r2=7788
0.315068493151
0.221917808219
0.364383561644
0.361643835616
953: RandomState r1=7231 and r2=1481
0.356164383562
0.197260273973
0.334246575342
0.33698630137
954: RandomState r1=7700 and r2=5378
0.347945205479
0.194520547945
0.372602739726
0.353424657534
955: RandomState r1=7189 and r2=4755
0.312328767123
0.158904109589
0.347945205479
0.320547945205
956: RandomState r1=5630 and r2=8171
0.356164383562
0.197260273973
0.378082191781
0.315068493151
957: RandomState r1=883 and r2=5024
0.347945205479
0.21095890411
0.375342465753
0.32

In [21]:
print('Results with shadowing:')
1 - p

Results with shadowing:


array([ 0.6476137 ,  0.8070274 ,  0.62810137,  0.65159452])